In [ ]:
import os
import cv2
import numpy as np
import librosa
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

############ Custom parameters ##############
IMG_SIZE = 224  # Reduced image size for memory efficiency
BATCH_SIZE = 16  # Smaller batch size to avoid memory issues
EPOCHS = 5  # Number of training epochs
N_CLASSES = 2  # Number of output classes (real, fake)
N_FRAMES = 5  # Number of frames to extract per video (randomly selected or averaged)
#############################################

# Paths for data (replace with actual paths)
real_video_path = r"\\192.168.1.70\Cyber Security\lipsync\AVLips v1.0_impana\AVLips\output\real video"
real_audio_path = r"\\192.168.1.70\Cyber Security\lipsync\AVLips v1.0_impana\AVLips\output\real audio"
fake_video_path = r"\\192.168.1.70\Cyber Security\lipsync\AVLips v1.0_impana\AVLips\output\fake video"
fake_audio_path = r"\\192.168.1.70\Cyber Security\lipsync\AVLips v1.0_impana\AVLips\output\fake audio"

In [29]:
import cv2
import numpy as np

# Function to load multiple frames from a folder
def load_video_frames(video_folder, img_size, num_frames=10):
    """
    Load multiple video frames from a folder and resize them.
    
    Parameters:
    video_folder (str): The path to the folder containing the video frames.
    img_size (int): The size to which each frame should be resized (img_size x img_size).
    num_frames (int): The number of frames to load for each video (default: 10).
    
    Returns:
    numpy array: The array of stacked frames with shape (img_size, img_size, num_frames, 3).
    """
    frame_files = sorted(os.listdir(video_folder))[:num_frames]  # Get the first 'num_frames' frames
    frames = []
    
    for frame_file in frame_files:
        frame_path = os.path.join(video_folder, frame_file)
        frame = cv2.imread(frame_path)
        if frame is not None:
            frame = cv2.resize(frame, (img_size, img_size))  # Resize to img_size x img_size
            frames.append(frame)

    # If there are less than 'num_frames', repeat the last frame to maintain consistency
    while len(frames) < num_frames:
        frames.append(frames[-1])

    # Stack frames along the third axis (time dimension), resulting in shape (img_size, img_size, num_frames, 3)
    return np.stack(frames, axis=2)


In [33]:
def load_audio_mfcc(mfcc_file):
    """Load the MFCC features from the .npy file."""
    if os.path.exists(mfcc_file):
        return np.load(mfcc_file)
    else:
        raise FileNotFoundError(f"MFCC file {mfcc_file} not found")


In [35]:
def process_and_train_model(real_video_path, fake_video_path, real_audio_path, fake_audio_path, img_size, batch_size, epochs):
    # Initialize the model
    input_shape_video = (img_size, img_size, N_FRAMES, 3)  # Adjust for the number of frames
    input_shape_audio = (64, 100, 1)  # Adjust based on your model's MFCC shape
    model = build_multimodal_model(input_shape_video, input_shape_audio)  # Both input shapes provided
    model.summary()

    real_video_folders = sorted(os.listdir(real_video_path))
    fake_video_folders = sorted(os.listdir(fake_video_path))

    for i in tqdm(range(0, len(real_video_folders), batch_size)):
        # Prepare batches
        real_batch = real_video_folders[i:i + batch_size]
        fake_batch = fake_video_folders[i:i + batch_size]
        
        real_images, real_labels = [], []
        fake_images, fake_labels = [], []

        # Load real video and audio batches
        for real_folder in real_batch:
            try:
                video_frames = load_video_frames(os.path.join(real_video_path, real_folder), img_size, n_frames=N_FRAMES)
                mfcc_file = os.path.join(real_audio_path, f"{real_folder}_mfcc.npy")
                mfcc_features = load_audio_mfcc(mfcc_file)
                combined = combine_image_and_audio(video_frames, mfcc_features, img_size)
                real_images.append(combined)
                real_labels.append(0)  # 0 for real
            except Exception as e:
                print(f"Error processing real folder {real_folder}: {e}")
        
        # Load fake video and audio batches
        for fake_folder in fake_batch:
            try:
                video_frames = load_video_frames(os.path.join(fake_video_path, fake_folder), img_size, n_frames=N_FRAMES)
                mfcc_file = os.path.join(fake_audio_path, f"{fake_folder}_mfcc.npy")
                mfcc_features = load_audio_mfcc(mfcc_file)
                combined = combine_image_and_audio(video_frames, mfcc_features, img_size)
                fake_images.append(combined)
                fake_labels.append(1)  # 1 for fake
            except Exception as e:
                print(f"Error processing fake folder {fake_folder}: {e}")

        # Combine real and fake data
        X = np.concatenate((real_images, fake_images), axis=0)
        y = np.concatenate((real_labels, fake_labels), axis=0)

        # Normalize data
        X = X / 255.0

        # One-hot encode labels
        y = to_categorical(y, N_CLASSES)

        # Split into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

        # Train the model on each batch
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

    return model


In [37]:
def combine_image_and_audio(video_frames, audio_mfcc, img_size):
    """
    Combine video frames (image) and audio MFCC into a single input.
    - video_frames: shape (n_frames, img_size, img_size, 3)
    - audio_mfcc: shape (n_mfcc, timesteps)
    """
    # Ensure that video_frames has 4 dimensions (n_frames, img_size, img_size, 3)
    if len(video_frames.shape) == 3:
        video_frames = np.expand_dims(video_frames, axis=0)  # Add frame dimension if not present

    # Resize the MFCC to match the image size (img_size x img_size)
    resized_mfcc = cv2.resize(audio_mfcc, (img_size, img_size))  # Resize to match image size

    # Expand the MFCC to have an extra channel (since MFCC is single-channel)
    resized_mfcc = np.expand_dims(resized_mfcc, axis=-1)  # Now shape: (img_size, img_size, 1)

    # Expand dimensions to add the frame dimension to MFCC, making it (1, img_size, img_size, 1)
    resized_mfcc = np.expand_dims(resized_mfcc, axis=0)

    # Now we need to tile the MFCC to match the number of frames in video_frames
    mfcc_tiled = np.tile(resized_mfcc, (video_frames.shape[0], 1, 1, 1))  # Tile across all video frames

    # Check the shapes before concatenation
    print(f"Video frames shape: {video_frames.shape}")
    print(f"Tiled MFCC shape: {mfcc_tiled.shape}")

    # Combine video and MFCC along the channel dimension (last axis)
    combined = np.concatenate([video_frames, mfcc_tiled], axis=-1)  # Concatenate along the channel axis

    return combined


In [39]:
import numpy as np

# Function to pad or truncate MFCC features to a fixed shape
def pad_or_truncate_mfcc(mfcc, desired_shape=(64, 100)):
    """
    Pads or truncates the MFCC array to the desired shape.
    
    Parameters:
    mfcc (numpy array): The input MFCC array.
    desired_shape (tuple): The desired output shape (e.g., (64, 100)).
    
    Returns:
    numpy array: The MFCC array padded or truncated to the desired shape.
    """
    mfcc_shape = mfcc.shape
    
    # Initialize the padded MFCC array with zeros
    padded_mfcc = np.zeros(desired_shape)  
    
    # Get the valid range (min of desired shape and actual MFCC shape)
    min_shape = np.minimum(mfcc_shape, desired_shape)
    
    # Copy the valid part of the MFCC array into the padded_mfcc
    padded_mfcc[:min_shape[0], :min_shape[1]] = mfcc[:min_shape[0], :min_shape[1]]
    
    return padded_mfcc


In [41]:
import os
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

# Function to build the multimodal model
def build_multimodal_model(input_shape_video, input_shape_audio):
    video_input = keras.Input(shape=input_shape_video, name='video_input')
    audio_input = keras.Input(shape=input_shape_audio, name='audio_input')

    # Example: Build a simple CNN for video
    x = keras.layers.Conv3D(32, (3, 3, 3), activation='relu')(video_input)
    x = keras.layers.MaxPooling3D((2, 2, 2))(x)
    x = keras.layers.Flatten()(x)

    # Build a simple model for audio using 2D CNN
    y = keras.layers.Conv2D(32, (3, 3), activation='relu')(audio_input)  # Change to Conv2D
    y = keras.layers.MaxPooling2D((2, 2))(y)
    y = keras.layers.Flatten()(y)

    # Merge the two branches
    combined = keras.layers.concatenate([x, y])

    # Example: Fully connected layer
    z = keras.layers.Dense(64, activation='relu')(combined)
    z = keras.layers.Dense(1, activation='sigmoid')(z)  # Output layer

    # Create model
    model = keras.Model(inputs=[video_input, audio_input], outputs=z)

    return model

def process_and_train_model(real_video_path, fake_video_path, real_audio_path, fake_audio_path, img_size, batch_size, epochs):
    # Initialize the model
    input_shape_video = (img_size, img_size, 10, 3)
    input_shape_audio = (64, 100, 1)
    model = build_multimodal_model(input_shape_video, input_shape_audio)
    
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    real_video_folders = sorted(os.listdir(real_video_path))
    fake_video_folders = sorted(os.listdir(fake_video_path))

    for i in tqdm(range(0, len(real_video_folders), batch_size)):
        real_batch = real_video_folders[i:i + batch_size]
        fake_batch = fake_video_folders[i:i + batch_size]
        
        real_videos, real_audios, real_labels = [], [], []
        fake_videos, fake_audios, fake_labels = [], [], []

        for real_folder in real_batch:
            video_frames = load_video_frames(os.path.join(real_video_path, real_folder), img_size, num_frames=10)
            mfcc_file = os.path.join(real_audio_path, f"{real_folder}_mfcc.npy")
            mfcc_features = load_audio_mfcc(mfcc_file)
            mfcc_features = pad_or_truncate_mfcc(mfcc_features, desired_shape=(64, 100))

            real_videos.append(video_frames)
            real_audios.append(mfcc_features)
            real_labels.append(0)

        for fake_folder in fake_batch:
            video_frames = load_video_frames(os.path.join(fake_video_path, fake_folder), img_size, num_frames=10)
            mfcc_file = os.path.join(fake_audio_path, f"{fake_folder}_mfcc.npy")
            mfcc_features = load_audio_mfcc(mfcc_file)
            mfcc_features = pad_or_truncate_mfcc(mfcc_features, desired_shape=(64, 100))

            fake_videos.append(video_frames)
            fake_audios.append(mfcc_features)
            fake_labels.append(1)

        X_video = np.stack(real_videos + fake_videos, axis=0)
        X_audio = np.stack(real_audios + fake_audios, axis=0)
        y = np.array(real_labels + fake_labels)  # Use binary labels

        X_video = X_video / 255.0
        X_audio = np.expand_dims(X_audio, -1)

        X_video_train, X_video_val, X_audio_train, X_audio_val, y_train, y_val = train_test_split(
            X_video, X_audio, y, test_size=0.2, stratify=y, random_state=42
        )

        model.fit([X_video_train, X_audio_train], y_train, epochs=epochs, batch_size=batch_size, validation_data=([X_video_val, X_audio_val], y_val))

    return model
if __name__ == "__main__":
    trained_model = process_and_train_model(real_video_path, fake_video_path, real_audio_path, fake_audio_path, IMG_SIZE, BATCH_SIZE, EPOCHS)


  0%|                                                                                          | 0/213 [00:00<?, ?it/s]

Epoch 1/5


C:\Users\Dell\anaconda3\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['video_input', 'audio_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.4867 - loss: 42.4432 - val_accuracy: 0.5714 - val_loss: 15.8763
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.4392 - loss: 46.9375 - val_accuracy: 0.4286 - val_loss: 152.1008
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.5133 - loss: 122.0027 - val_accuracy: 0.4286 - val_loss: 23.0051
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6617 - loss: 10.0080 - val_accuracy: 0.5714 - val_loss: 47.4428
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.4658 - loss: 52.5878 - val_accuracy: 0.5714 - val_loss: 54.0350


  0%|▍                                                                               | 1/213 [00:28<1:39:05, 28.05s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████████▎| 211/213 [1:28:22<00:48, 24.04s/it]

Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6083 - loss: 1.7006 - val_accuracy: 0.7143 - val_loss: 0.7029
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7567 - loss: 0.9626 - val_accuracy: 0.5714 - val_loss: 1.4928
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6825 - loss: 1.6689 - val_accuracy: 0.7143 - val_loss: 1.4460
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7092 - loss: 1.0654 - val_accuracy: 0.8571 - val_loss: 0.3999
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.8100 - loss: 0.4292 - val_accuracy: 0.5714 - val_loss: 0.5565


100%|█████████████████████████████████████████████████████████████████████████████▋| 212/213 [1:28:46<00:23, 23.82s/it]

Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.8125 - loss: 0.4924 - val_accuracy: 1.0000 - val_loss: 0.0366
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.9375 - loss: 0.4930 - val_accuracy: 1.0000 - val_loss: 0.1151
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.9375 - loss: 0.5205 - val_accuracy: 1.0000 - val_loss: 0.1621
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.9375 - loss: 0.4696 - val_accuracy: 1.0000 - val_loss: 0.1208
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.9375 - loss: 0.3362 - val_accuracy: 1.0000 - val_loss: 0.0506


100%|██████████████████████████████████████████████████████████████████████████████| 213/213 [1:29:00<00:00, 25.07s/it]


In [42]:
trained_model.save("abcde.h5")

In [59]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("abcd.h5")

# Function to load multiple frames from a folder (same as in your training code)
def load_video_frames(video_folder, img_size, num_frames=10):
    frame_files = sorted(os.listdir(video_folder))[:num_frames]  # Get the first 'num_frames' frames
    frames = []

    for frame_file in frame_files:
        frame_path = os.path.join(video_folder, frame_file)
        frame = cv2.imread(frame_path)
        if frame is not None:
            frame = cv2.resize(frame, (img_size, img_size))  # Resize to img_size x img_size
            frames.append(frame)

    # If there are less than 'num_frames', repeat the last frame to maintain consistency
    while len(frames) < num_frames:
        frames.append(frames[-1])

    # Stack frames along the third axis (time dimension), resulting in shape (img_size, img_size, num_frames, 3)
    return np.stack(frames, axis=2)

# Function to load MFCC features from a .npy file
def load_audio_mfcc(mfcc_file):
    if os.path.exists(mfcc_file):
        return np.load(mfcc_file)
    else:
        raise FileNotFoundError(f"MFCC file {mfcc_file} not found")

# Function to pad or truncate MFCC features to the fixed shape used in training
def pad_or_truncate_mfcc(mfcc, desired_shape=(64, 100)):
    mfcc_shape = mfcc.shape
    padded_mfcc = np.zeros(desired_shape)
    min_shape = np.minimum(mfcc_shape, desired_shape)
    padded_mfcc[:min_shape[0], :min_shape[1]] = mfcc[:min_shape[0], :min_shape[1]]
    return padded_mfcc

# Prediction function
def predict_real_or_fake(video_folder, mfcc_file, img_size, num_frames=10):
    # Load and process video frames
    video_frames = load_video_frames(video_folder, img_size, num_frames)
    video_frames = video_frames / 255.0  # Normalize to match the model's input during training
    video_frames = np.expand_dims(video_frames, axis=0)  # Add batch dimension

    # Load and process MFCC features
    mfcc_features = load_audio_mfcc(mfcc_file)
    mfcc_features = pad_or_truncate_mfcc(mfcc_features, desired_shape=(64, 100))
    mfcc_features = np.expand_dims(mfcc_features, axis=-1)  # Add channel dimension
    mfcc_features = np.expand_dims(mfcc_features, axis=0)  # Add batch dimension

    # Make a prediction
    prediction = model.predict([video_frames, mfcc_features])

    # Interpret the prediction
    if prediction >= 0.5:
        return "Fake"
    else:
        return "Real"

# Example usage (provide paths to new video and corresponding MFCC file)
video_folder_path = r"\\192.168.1.70\Cyber Security\lipsync\AVLips v1.0_impana\AVLips\output\fake video\1"
mfcc_file_path =r"\\192.168.1.70\Cyber Security\lipsync\AVLips v1.0_impana\AVLips\output\fake audio\1_mfcc.npy"

# Perform prediction
result = predict_real_or_fake(video_folder_path, mfcc_file_path, img_size=224, num_frames=10)
print(f"The given video and audio are: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
The given video and audio are: Fake


In [91]:
import numpy as np
from tensorflow import keras

def load_and_preprocess_data(video_path, audio_path, img_size, num_frames=10):
    """
    Load video frames and audio MFCC features, and preprocess them for prediction.
    """
    # Load video frames
    video_frames = load_video_frames(video_path, img_size, num_frames=num_frames)
    
    # Load audio MFCC features
    mfcc_features = load_audio_mfcc(audio_path)
    mfcc_features = pad_or_truncate_mfcc(mfcc_features, desired_shape=(64, 100))  # Ensure shape consistency

    # Expand dimensions for audio if needed (e.g., add a channel dimension)
    mfcc_features = np.expand_dims(mfcc_features, axis=-1)  # Shape: (64, 100, 1)

    return video_frames, mfcc_features

def predict(model, video_path, audio_path, img_size, num_frames=10):
    """
    Predict if the given video and audio are real or fake.
    """
    # Load and preprocess the data
    video_frames, mfcc_features = load_and_preprocess_data(video_path, audio_path, img_size, num_frames)

    # Normalize video frames
    video_frames = video_frames / 255.0  # Normalize to [0, 1]

    # Prepare the input arrays for prediction
    video_input = np.expand_dims(video_frames, axis=0)  # Shape: (1, img_size, img_size, num_frames, 3)
    audio_input = np.expand_dims(mfcc_features, axis=0)  # Shape: (1, 64, 100, 1)

    # Make prediction
    prediction = model.predict([video_input, audio_input])

    # Output: If using sigmoid, the output is a probability; if softmax, get the class with max probability
    if prediction.shape[1] == 1:  # Sigmoid output
        return "Fake" if prediction[0][0] >= 0.5 else "Real"
    else:  # Softmax output
        predicted_class = np.argmax(prediction, axis=-1)
        return "Fake" if predicted_class[0] == 1 else "Real"

if __name__ == "__main__":
    # Example usage
    model = keras.models.load_model('abcd.h5')  # Load your trained model
    video_path = r"\\192.168.1.70\Cyber Security\lipsync\AVLips v1.0_impana\AVLips\output\real video\214"
    audio_path = r"\\192.168.1.70\Cyber Security\lipsync\AVLips v1.0_impana\AVLips\output\real audio\214_mfcc.npy"
    img_size = 224  # Your image size
    result = predict(model, video_path, audio_path, img_size)
    print(f'The video is predicted to be: {result}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
The video is predicted to be: Real


In [115]:
import os
import cv2
import numpy as np
import librosa
from tensorflow.keras.models import load_model
import moviepy.editor as mp

# Load the saved model
model = load_model("abcd.h5")

# Function to extract frames from video
def extract_video_frames(video_file, img_size, num_frames=10):
    """
    Extracts frames from the input video file and resizes them.
    - video_file: path to the video file
    - img_size: desired size of the output frames (img_size x img_size)
    - num_frames: number of frames to extract
    """
    cap = cv2.VideoCapture(video_file)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_idxs = np.linspace(0, total_frames - 1, num_frames, dtype=int)  # Select evenly spaced frames

    for idx in frame_idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (img_size, img_size))  # Resize to img_size x img_size
            frames.append(frame)

    cap.release()

    # Stack frames along the time dimension (num_frames, img_size, img_size, 3)
    frames = np.stack(frames, axis=0)

    # Reorder dimensions from (num_frames, img_size, img_size, 3) to (img_size, img_size, num_frames, 3)
    frames = np.transpose(frames, (1, 2, 0, 3))

    return frames

# Function to extract MFCC features from video audio
def extract_audio_mfcc(video_file, desired_shape=(64, 100)):
    """
    Extracts MFCC features from the audio of the video file.
    - video_file: path to the video file
    - desired_shape: shape to which the MFCC features will be padded/truncated
    """
    # Extract audio from video
    video = mp.VideoFileClip(video_file)
    audio = video.audio

    # Convert audio to a temporary WAV file
    audio_path = "temp_audio.wav"
    audio.write_audiofile(audio_path, codec='pcm_s16le')  # PCM 16-bit WAV file

    # Load audio using librosa
    y, sr = librosa.load(audio_path, sr=None)

    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=64)

    # Pad or truncate the MFCC features to the desired shape
    mfcc = pad_or_truncate_mfcc(mfcc, desired_shape=desired_shape)

    # Clean up temporary audio file
    os.remove(audio_path)

    return mfcc

# Function to pad or truncate MFCC features (same as before)
def pad_or_truncate_mfcc(mfcc, desired_shape=(64, 100)):
    mfcc_shape = mfcc.shape
    padded_mfcc = np.zeros(desired_shape)
    min_shape = np.minimum(mfcc_shape, desired_shape)
    padded_mfcc[:min_shape[0], :min_shape[1]] = mfcc[:min_shape[0], :min_shape[1]]
    return padded_mfcc

# Updated prediction function
def predict_real_or_fake(video_file, img_size, num_frames=10):
    # Extract video frames from the video file
    video_frames = extract_video_frames(video_file, img_size, num_frames)
    video_frames = video_frames / 255.0  # Normalize to match the model's input during training
    video_frames = np.expand_dims(video_frames, axis=0)  # Add batch dimension

    # Extract MFCC features from the audio of the video file
    mfcc_features = extract_audio_mfcc(video_file, desired_shape=(64, 100))
    mfcc_features = np.expand_dims(mfcc_features, axis=-1)  # Add channel dimension
    mfcc_features = np.expand_dims(mfcc_features, axis=0)  # Add batch dimension

    # Make a prediction
    prediction = model.predict([video_frames, mfcc_features])

    # Print the raw prediction
    print("Raw prediction:", prediction)

    # Interpret the prediction
    if prediction >= 0.5:
        return "Fake"
    else:
        return "Real"


# Example usage (provide the path to your new video file with audio)
video_file_path =r"C:\Users\Dell\Downloads\WhatsApp Video 2024-10-24 at 2.57.29 PM.mp4"
# Perform prediction
result = predict_real_or_fake(video_file_path, img_size=224, num_frames=10)
print(f"The given video and audio are: {result}")


MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Raw prediction: [[0.9569234]]
The given video and audio are: Fake
